In [1]:
import requests
import json
import pandas as pd
import os
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
import re
import numpy as np
import logging
import sys
import time
from datetime import timedelta

USER_AGENT environment variable not set, consider setting it to identify your requests.


Testing the LLM client connection

In [2]:
# imports + project path
import os
import sys
from pathlib import Path


# Go up one folder from /notebooks to the repo root.
# So here its /Users/andrewlim/Documents/Phosho_Pipeline
PROJECT_ROOT = Path.cwd().resolve().parent  # if notebook is in /notebooks
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Testing llm_client function
from phospho.llm_normalize import get_llm_client

In [3]:
# build client (smoke test)
client = get_llm_client()
print(type(client))


<class 'langchain_openai.chat_models.azure.AzureChatOpenAI'>


In [4]:
# API call test
# This block confirms auth + endpoint + deployment are valid.
try:
    # Send a test prompt to the model w/ configured client and store response in variable 
    resp = client.invoke("Reply with exactly: OK")
    print(resp.content)
except Exception as e:
    # Catch and print any errors (error class name + message)
    print(type(e).__name__, str(e))

OK


In [5]:
# Testing the parse_interactions function 
from phospho.llm_normalize import parse_interactions

# Example simulated LLM output with valid lines + blanks + N/A
sample_raw = """
AKT1(kinase), FOXO3(substrate), Ser-253(location), PubMed:12345
N/A
MAPK1(kinase), ELK1(substrate), Ser-383(location), PubMed:67890
"""

parsed = parse_interactions(sample_raw)

print("Parsed records:", len(parsed))
for i, row in enumerate(parsed, 1):
    print(i, row)


Parsed records: 2
1 {'raw': 'AKT1(kinase), FOXO3(substrate), Ser-253(location), PubMed:12345'}
2 {'raw': 'MAPK1(kinase), ELK1(substrate), Ser-383(location), PubMed:67890'}


In [ ]:
# Ensure repo root is importable
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from phospho.llm_normalize import normalize_ptm_texts

# Toy PTM inputs
ptm_texts = [
    "Phosphorylated at Ser-473 by MTORC2.",
    "Phosphorylated by AKT1 at Thr-308 (PubMed:12345678).",
]
substrate_gene = "AKT1"

# Run normalization
outputs = normalize_ptm_texts(ptm_texts, substrate_gene)

print(outputs)

# print("Num outputs:", len(outputs))
# for i, out in enumerate(outputs, 1):
#     print(f"\n[{i}] {out}")

['MTORC2(kinase), AKT1(substrate), Ser-473(location), N/A', 'AKT1(kinase), AKT1(substrate), Thr-308(location), PubMed:12345678']
